<a href="https://colab.research.google.com/github/MinsuChoKW/Data-Analytics-2025/blob/main/week4_event_logs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📘 Week 4 - 이벤트 로그 · 프로세스 마이닝 (PM4Py)

- 이 노트북은 **최신 PM4Py** 버전(권장: 2.7 이상) 기준으로 작성되었다.  
- 실습 항목: 이벤트 로그 로드, 간단 전처리, Dotted Chart 등 확인
- 오류가 발생하면 아래 환경 점검 셀을 먼저 실행해 버전을 확인하고, 필요 시 pip 설치 주석을 해제하여라.

In [1]:
# ✅ 환경 점검: PM4Py 버전 확인 (필요 시 설치)
try:
    import pm4py
    print("pm4py version:", pm4py.__version__)
except Exception as e:
    print("pm4py가 설치되어 있지 않거나 환경 문제가 있습니다.")
    print("Colab/로컬에서 아래 주석을 해제해 설치하세요:")
    print("!pip install -U pm4py")

pm4py version: 2.7.11.9


## 환경 설정 (Setup)

본 노트북에서는 아래의 라이브러리를 활용한다:

* [PM4Py](https://pm4py.fit.fraunhofer.de/)
* [pandas](https://pandas.pydata.org/)
* [matplotlib](https://matplotlib.org)


In [2]:
import pandas as pd
import pm4py
import matplotlib.pyplot as plt

## 이벤트 로그 (Event Logs)
이 부분에서는 이벤트 로그와 프로세스 마이닝 방법의 기반이 되는 고유한 속성을 소개한다.. 본 실습을 위해서는 `data` 디렉토리에 있는 CSV 파일에서 로드해야 한다. 이 강의에서는 다음 데이터셋을 사용할 것이다:

* Patients: 병원 환경에서 합성적으로 생성된 예제 이벤트 로그이다.
* Sepsis: 네덜란드 병원에서 가져온 실제 이벤트 로그이다. 이 이벤트 로그는 다음에서 공개적으로 사용할 수 있으며 많은 프로세스 마이닝 관련 출판물에서 사용되었다: https://doi.org/10.4121/uuid:915d2bfb-7e84-49ad-a286-dc35f063a460

### Import Patients Data

In [6]:
patients = pd.read_csv("../data/patients.csv", sep=';')
patients['time'] = pd.to_datetime(patients['time'])
num_rows = len(patients)
print("Number of rows: {}".format(num_rows))

FileNotFoundError: [Errno 2] No such file or directory: '../data/patients.csv'

### Import Sepsis Data

In [7]:
sepsis = pd.read_csv("../data/sepsis.csv", sep=';')
num_rows = len(sepsis)
sepsis['timestamp'] = pd.to_datetime(sepsis['timestamp'])
print("Number of rows: {}".format(num_rows))

FileNotFoundError: [Errno 2] No such file or directory: '../data/sepsis.csv'

### Exploring Event Data

우선 이벤트 로그의 구조나 특성에 대한 사전 지식 없이 이벤트 데이터를 탐색해 본다. 이를 위해 표준 Pandas와 Matplotlib를 활용한다.

데이터 내 Timestamp의 포멧 변경을 위하여 아래의 항목을 적용한다.

In [ ]:
patients['time'] = pd.to_datetime(patients['time'], utc=True, errors='coerce').dt.tz_convert(None)
patients['time'] = patients['time'].astype('datetime64[ns]')

In [ ]:
sepsis['timestamp'] = pd.to_datetime(sepsis['timestamp'], utc=True, errors='coerce').dt.tz_convert(None)
sepsis['timestamp'] = sepsis['timestamp'].astype('datetime64[ns]')

In [ ]:
sepsis.head()

이벤트 로그에서 가장 중요한 요소는 `time` 이다. 이를 통해 이벤트들의 순서를 확립할 수 있다.

In [ ]:
t = pd.to_datetime(patients['time'])

x = t[t < pd.Timestamp('2017-01-31')].dropna()

plt.figure(figsize=(10, 2))
plt.scatter(x, [0]*len(x), s=10)
plt.ylabel("Event")
plt.yticks([])
plt.xticks(rotation=45)
plt.grid(axis='x', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

또한 어떤 종류의 행동이나 `활동(activities)`이 수행되었는지에 대한 정보도 필요하다.

In [ ]:
patients.drop_duplicates(subset='handling')[["handling"]]

다른 어떤 데이터가 있는지 한번 살펴보자.

In [ ]:
patients.drop_duplicates(subset='patient')[["patient"]].head()

아마도 환자 식별자가 프로세스의 `케이스(case)`를 정의하는 데 좋은 후보가 될 수 있을 것이다. 이는 우리가 추적하고자 하는 '엔티티'이기 때문이다. 개별 환자별로 발생한 이벤트 수를 세어보면, 각 환자마다 유사한 이벤트 개수가 나타나는데, 이는 일반적으로 프로세스 케이스 식별자를 정하는 데 좋은 지표가 된다.

In [ ]:
patients.groupby(['patient'])["patient"].agg(['count']).head()

환자 식별자를 `케이스 식별자(case identifier)`로 삼아 프로세스를 따라가기로 하자.

In [ ]:
patients['time'] = pd.to_datetime(patients['time'])
patients_sample = patients[patients['time'] < pd.Timestamp('2017-01-31')].dropna(subset=['time'])

plt.figure(figsize=(10, 4))
scatter = plt.scatter(
    patients_sample['time'],
    patients_sample['patient'],
    c=pd.Categorical(patients_sample['handling']).codes,
    cmap='tab10',
    s=20
)

plt.ylabel("Patient")
plt.xticks(rotation=45)
plt.grid(axis='x', linestyle='--', alpha=0.5)

handles, labels = scatter.legend_elements(prop="colors", alpha=0.6)
plt.legend(handles, pd.Categorical(patients_sample['handling']).categories,
           title="Handling", bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()

위 산점도는 프로세스 마이닝 분야에서 `도트 차트(Dotted Chart)`라고 불리며, 케이스 단위로 묶었을 때 이벤트와 그 시간적 관계를 한눈에 보여준다. 각 이벤트 시퀀스(`트레이스`라고도 함)는 `등록(Registration)` 이벤트로 시작하는 것처럼 보인다. 이제 이벤트 데이터를 환자 식별자와 시간 기준으로 정렬해서 살펴보자.

In [ ]:
patients.sort_values(['patient', 'time']).head(14)

개별 프로세스 실행(예: 환자 1의 경우)은 여러 활동이 순차적으로 수행되는 것으로 구성된다. 하지만 단순히 이벤트의 순서만 있는 것이 아니라 더 많은 정보가 주어진다. 각 활동의 발생에는 두 개의 이벤트가 있는데, 하나는 `start` 이벤트이고 다른 하나는 `complete` 이벤트이며 이는 `registration_type` 열에 기록된다. 이러한 이벤트들은 활동의 라이프사이클을 나타내며, 활동의 `지속 시간(duration)`을 측정할 수 있게 한다.

### Further resources

* [XES Standard](http://xes-standard.org/)
* [Importing XES event logs](https://processintelligence.solutions/pm4py/api?page=pm4py.html%23pm4py.read.read_xes)

####성찰 질문(Reflection Questions)
* 왜 이 데이터셋에 `.order`라는 열이 포함되어 있을까?
* `employee` 열은 어떻게 활용할 수 있을까?
* `handling_id` 열은 어떤 용도로 쓰이며, 어떤 상황에서 필요할까?

## Basic Process Visualization

### Dotted Chart

`Dotted Chart`는 각 트레이스의 시간적 측면을 더해주고, 모든 트레이스를 한눈에 시각화한다. 다양한 방식으로 설정할 수 있어 프로세스 동작의 시간 관련 특성을 이해하는 데 유용하다. PM4Py는 기본적인 Dotted Chart 시각화를 제공한다.

In [ ]:
patients_log = pm4py.format_dataframe(patients, case_id='patient', activity_key='handling', timestamp_key='time')
pm4py.view_dotted_chart(pm4py.filter_time_range(patients_log, "1970-01-01 00:00:00", "2017-01-31 00:00:00", mode='events'))

대안적으로, 시각화를 더 세밀하게 커스터마이즈하기 위해 동일한 뷰를 plt를 사용해 간단히 재현할 수 있으며, 이를 통해 관점 선택에서 더 큰 유연성을 가질 수 있다.

In [ ]:
patients_sorted = patients.sort_values(['time'])
patients_sorted['patient'] = pd.Categorical(patients_sorted['patient'],
                                            categories = patients_sorted['patient'].drop_duplicates().tolist()[::-1], ordered= True)

#### Absolute Time Dimension

In [ ]:
df = patients_sorted.copy()
df['time'] = pd.to_datetime(df['time'], utc=True, errors='coerce').dt.tz_convert(None)

df = df[df['time'] < pd.Timestamp('2017-01-31')].dropna(subset=['time'])

plt.figure(figsize=(10, 4))
scatter = plt.scatter(
    df['time'],
    df['patient'],
    c=pd.Categorical(df['handling']).codes,
    cmap='tab10',
    s=20
)

plt.xticks(rotation=45, ha='right')
plt.ylabel("Patient")
plt.yticks([])
plt.grid(axis='x', linestyle='--', alpha=0.4)

handles, labels = scatter.legend_elements(prop="colors", alpha=0.6)
plt.legend(handles,
           pd.Categorical(df['handling']).categories,
           title="Handling",
           bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()

#### Relative Time Dimension

시간을 상대값으로 바꿔서 그 결과를 담을 새 열 `time_relative`를 추가한다.

In [ ]:
patients_sorted['time_relative'] = patients_sorted['time'].sub( patients_sorted.groupby('patient')['time'].transform('first'))

In [ ]:
df = patients_sorted.copy()

plt.figure(figsize=(10, 4))
scatter = plt.scatter(
    df['time_relative'],
    df['patient'],
    c=pd.Categorical(df['handling']).codes,
    cmap='tab10',
    s=20
)

plt.xticks(rotation=45, ha='right')
plt.ylabel("Patient")
plt.yticks([])
plt.grid(axis='x', linestyle='--', alpha=0.4)

handles, labels = scatter.legend_elements(prop="colors", alpha=0.6)
plt.legend(handles,
           pd.Categorical(df['handling']).categories,
           title="Handling",
           bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()

각 케이스별 전체 소요시간을 위한 새 열 `duration`을 추가한다.

In [ ]:
patients_sorted['duration'] = patients_sorted.groupby('patient')['time_relative'].transform('max')

In [ ]:
df = patients_sorted.sort_values(['duration']).copy()
df['patient'] = pd.Categorical(
    df['patient'],
    categories=df['patient'].drop_duplicates().tolist()[::-1],
    ordered=True
)

x = pd.to_timedelta(df['time_relative'], errors='coerce')
mask = x.notna()
x_days = x[mask].dt.total_seconds() / 86400.0

plt.figure(figsize=(10, 6))
scatter = plt.scatter(
    x_days,
    df.loc[mask, 'patient'].astype(str),
    c=pd.Categorical(df.loc[mask, 'handling']).codes,
    cmap='tab10',
    s=20
)

plt.xlabel("time_relative (days)")
plt.ylabel("Patient")
plt.yticks([])
plt.xticks(rotation=45, ha='right')
plt.grid(axis='x', linestyle='--', alpha=0.4)

handles, _ = scatter.legend_elements(prop="colors", alpha=0.6)
plt.legend(handles,
           pd.Categorical(df.loc[mask, 'handling']).categories,
           title="Handling",
           bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()

PM4Py에서 제공하는 다른 기본적인 프로세스 시각화 옵션.

* [Basic Process Statistics](https://processintelligence.solutions/pm4py/api?page=pm4py.html%23module-pm4py.stats)

## Process Map Visualization

마찬가지로, PM4Py에는 내장된 선행 관계 행렬(Precedence Matrix) 시각화 기능이 없지만, 이를 손쉽게 재현할 수 있다.

In [ ]:
patients_sorted['antecedent'] = patients_sorted.groupby(["patient"])['handling'].shift(1).fillna("Start")
patients_sorted['consequent'] = patients_sorted['handling']

In [ ]:
transition_counts = patients_sorted.groupby(['antecedent', 'consequent']).size().reset_index(name='count')
heatmap_data = transition_counts.pivot(index='antecedent', columns='consequent', values='count').fillna(0)

fig, ax = plt.subplots(figsize=(12, 8))
cax = ax.matshow(heatmap_data, cmap='Blues')

ax.set_title('Directly-Follows Heatmap (Patients Data)', y=1.05)
ax.set_xlabel('Consequent Activity')
ax.set_ylabel('Antecedent Activity')

ax.set_xticks(np.arange(len(heatmap_data.columns)))
ax.set_xticklabels(heatmap_data.columns, rotation=45, ha='right')
ax.set_yticks(np.arange(len(heatmap_data.index)))
ax.set_yticklabels(heatmap_data.index)

ax.xaxis.tick_bottom()

fig.colorbar(cax, ax=ax, orientation='vertical', label='Frequency')

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

### Directly-follows Graph / Process Map

아래는 다음 구현에서 자세히 다룰 Process Discovery에 한 예인 DFG를 나타낸다.

In [ ]:
patients_log = patients_log[patients_log['registration_type'] == 'complete']

In [ ]:
dfg, sa, ea = pm4py.discover_directly_follows_graph(patients_log)

In [ ]:
pm4py.view_dfg(dfg, sa, ea)

In [ ]:
from pm4py.algo.discovery.dfg import algorithm as dfg_discovery
from pm4py.algo.discovery.dfg import algorithm as dfg_discovery
from pm4py.visualization.dfg import visualizer as dfg_visualization

dfg = dfg_discovery.apply(patients_log)

dfg = dfg_discovery.apply(patients_log, variant=dfg_discovery.Variants.PERFORMANCE)
gviz = dfg_visualization.apply(dfg, log=patients_log, variant=dfg_visualization.Variants.PERFORMANCE)
dfg_visualization.view(gviz)

## Real-life Processes

In [ ]:
sepsis_sorted = sepsis.sort_values(['timestamp'])
sepsis_sorted['timestamp'] = pd.to_datetime(sepsis_sorted['timestamp'])

In [ ]:
sepsis_sorted['antecedent'] = sepsis_sorted.groupby(["case_id"])['activity'].shift(1).fillna("Start")
sepsis_sorted['consequent'] = sepsis_sorted['activity']

In [ ]:
transition_counts = sepsis_sorted.groupby(['antecedent', 'consequent']).size().reset_index(name='count')
heatmap_data = transition_counts.pivot(index='antecedent', columns='consequent', values='count').fillna(0)

fig, ax = plt.subplots(figsize=(12, 8))
cax = ax.matshow(heatmap_data, cmap='Blues')

plt.title('Directly-Follows Heatmap (Sepsis Data)')
plt.xlabel('Consequent Activity')
plt.ylabel('Antecedent Activity')

ax.set_xticks(np.arange(len(heatmap_data.columns)))
ax.set_xticklabels(heatmap_data.columns, rotation=45, ha='right')
ax.set_yticks(np.arange(len(heatmap_data.index)))
ax.set_yticklabels(heatmap_data.index)

ax.xaxis.tick_bottom()

plt.xticks(ticks=range(len(heatmap_data.columns)), labels=heatmap_data.columns, rotation=45, ha='right')
plt.yticks(ticks=range(len(heatmap_data.index)), labels=heatmap_data.index, rotation=0)

fig.colorbar(cax, ax=ax, orientation='vertical', label='Frequency')

plt.tight_layout()
plt.show()
